### partie 2 suite : modèle XGB 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


import pandas as pd
import numpy as np

import sklearn 
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score, confusion_matrix

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

import joblib

In [6]:
data = pd.read_csv('get_around_pricing_project.csv', index_col=0)

In [7]:
data.shape

(4843, 14)

In [8]:
data.isnull().sum()

model_key                    0
mileage                      0
engine_power                 0
fuel                         0
paint_color                  0
car_type                     0
private_parking_available    0
has_gps                      0
has_air_conditioning         0
automatic_car                0
has_getaround_connect        0
has_speed_regulator          0
winter_tires                 0
rental_price_per_day         0
dtype: int64

In [9]:
data.head()

,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,Citroën,140411,100,diesel,black,convertible,True,True,False,False,True,True,True,106
1,Citroën,13929,317,petrol,grey,convertible,True,True,False,False,False,True,True,264
2,Citroën,183297,120,diesel,white,convertible,False,False,False,False,True,False,True,101
3,Citroën,128035,135,diesel,red,convertible,True,True,False,False,True,True,True,158
4,Citroën,97097,160,diesel,silver,convertible,True,True,False,False,False,True,True,183


In [11]:
list=['Mini','Mazda','Honda','Lexus','Alfa Romeo']
for i in range(len(list)) : 
    data.drop(data[(data['model_key'] ==list[i])].index,axis=0,inplace=True)

In [12]:
data.describe(include="all")

,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
count,4835,4.835000e+03,4835.000000,4835,4835,4835,4835,4835,4835,4835,4835,4835,4835,4835.000000
unique,23,NaN,NaN,4,10,8,2,2,2,2,2,2,2,NaN
top,Citroën,NaN,NaN,diesel,black,estate,True,True,False,False,False,False,True,NaN
freq,969,NaN,NaN,4636,1632,1606,2657,3831,3860,3878,2612,3671,4506,NaN
mean,NaN,1.409773e+05,128.770010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121.151396
std,NaN,6.023039e+04,38.570249,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.519991
min,NaN,-6.400000e+01,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.000000
25%,NaN,1.029135e+05,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.000000
50%,NaN,1.410890e+05,120.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,119.000000
75%,NaN,1.752260e+05,135.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,136.000000


In [14]:
data.dropna(inplace=True)

In [15]:
# Separate target variable Y from features X
print("Separating labels from features...")
target_variable = "rental_price_per_day"

X = data.drop(target_variable, axis = 1)
Y = data.loc[:,target_variable]

print("...Done.")
print()

print('Y : ')
print(Y.head())
print()
print('X :')
print(X.head())

Separating labels from features...
...Done.

Y : 
0    106
1    264
2    101
3    158
4    183
Name: rental_price_per_day, dtype: int64

X :
  model_key  mileage  engine_power    fuel paint_color     car_type  \
0   Citroën   140411           100  diesel       black  convertible   
1   Citroën    13929           317  petrol        grey  convertible   
2   Citroën   183297           120  diesel       white  convertible   
3   Citroën   128035           135  diesel         red  convertible   
4   Citroën    97097           160  diesel      silver  convertible   

   private_parking_available  has_gps  has_air_conditioning  automatic_car  \
0                       True     True                 False          False   
1                       True     True                 False          False   
2                      False    False                 False          False   
3                       True     True                 False          False   
4                       True     True     

In [17]:
# Automatically detect names of numeric/categorical columns
numeric_features = []
categorical_features = []
for i,t in X.dtypes.items():
    if ('float' in str(t)) or ('int' in str(t)) :
        numeric_features.append(i)
    else :
        categorical_features.append(i)

print('Found numeric features ', numeric_features)
print('Found categorical features ', categorical_features)

Found numeric features  ['mileage', 'engine_power']
Found categorical features  ['model_key', 'fuel', 'paint_color', 'car_type', 'private_parking_available', 'has_gps', 'has_air_conditioning', 'automatic_car', 'has_getaround_connect', 'has_speed_regulator', 'winter_tires']


In [18]:
numeric_features=['mileage', 'engine_power']
categorical_features = ['model_key', 'fuel', 'paint_color', 'car_type', 'private_parking_available', 'has_gps', 'has_air_conditioning', 'automatic_car', 'has_getaround_connect', 'has_speed_regulator', 'winter_tires']

In [19]:
# Divide dataset Train set & Test set 
print("Dividing into train and test sets...")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
print("...Done.")
print()

Dividing into train and test sets...
...Done.



## Preprocessing

In [20]:
# Create pipeline for numeric features
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

In [21]:
# Create pipeline for categorical features
categorical_transformer = Pipeline(
    steps=[
    ('encoder', OneHotEncoder(drop='first')) # first column will be dropped to avoid creating correlations between features
    ])

In [22]:
# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

##  le Gradient Boost Regressor

In [25]:
!pip install xgboost


  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/a2/48/d5da8591891327b0faf08179d420fba3893c6134bdd449497c5329e4cb01/xgboost-2.1.0-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB 435.7 kB/s eta 0:04:47
   ---------------------------------------- 0.2/124.9 MB 1.3 MB/s eta 0:01:36
   ---------------------------------------- 0.6/124.9 MB 3.8 MB/s eta 0:00:33
   ---------------------------------------- 1.2/124.9 MB 5.8 MB/s eta 0:00:22
    --------------------------------------- 1.7/124.9 MB 6.7 MB/s eta 0:00:19
    --------------------------------------- 2.2/124.9 MB 7.8 MB/s eta 0:00:16
    --------------------------------------- 2.8/124.9 MB 8.4 MB/s eta 0:00:15
   - -------------------------------------- 3.3/124.9 MB 8.7 MB/s eta 0:00:14
   - ----------------------

In [26]:
from xgboost import XGBRegressor


In [27]:
model = Pipeline(steps=[
        ("Preprocessing", preprocessor),
        ("Regressor",XGBRegressor(learning_rate= 0.05, max_depth= 8, min_child_weight= 4, n_estimators= 150))
    ])

In [28]:
model.fit(X_train, Y_train)

train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

print("r2 on train set : ", r2_score(Y_train, train_pred))
print("r2 on test set : ", r2_score(Y_test, test_pred))

r2 on train set :  0.9070636847816742
r2 on test set :  0.7674115318887376


### Tester le modèle localement

In [29]:
from joblib import dump, load
dump(model, 'model_xg.joblib')

['model_xg.joblib']

In [40]:
predictionFeatures = {
        "model_key" : "Renault",
        "mileage" : 14699,
        "engine_power" : 100,
        "fuel" : "diesel",
        "paint_color" : "black",
        "car_type" : "sedan",
        "private_parking_available" : False,
        "has_gps" : True,
        "has_air_conditioning" : False,
        "automatic_car" : False,
        "has_getaround_connect" : False,
        "has_speed_regulator" : True,
        "winter_tires" : True
        }

In [41]:
data = pd.DataFrame(dict(predictionFeatures), index=[10])
data

,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires
10,Renault,14699,100,diesel,black,sedan,False,True,False,False,False,True,True


In [42]:
loaded_model = load('model_xg.joblib')

In [43]:
prediction = loaded_model.predict(data)
response = {"predictions": prediction.tolist()[0]}

In [44]:
response

{'predictions': 146.4674072265625}